In [16]:
%%writefile create_db.hql

DROP DATABASE IF EXISTS stackoverflow CASCADE;
CREATE DATABASE stackoverflow LOCATION '/user/jovyan/store/stackoverflow';


Overwriting create_db.hql


In [17]:
%%writefile external_table.hql

USE stackoverflow;

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

DROP TABLE IF EXISTS posts_sample_external;

CREATE EXTERNAL TABLE posts_sample_external (
    id INT,
    post_type_id TINYINT,
    date STRING,
    owner_user_id INT,
    parent_id INT,
    score INT,
    favorite_count INT,
    tags STRING    
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' WITH SERDEPROPERTIES
('input.regex'='.*(?=\\bId=\"(\\d+)\")(?=.*\\bPostTypeId=\"(\\d+)\")(?=.*\\bCreationDate=\"(.{23})\")(?=.*\\bOwnerUserId=\"(\\d+)\")?(?=.*\\bParentId=\"(\\d+)\")?(?=.*\\bScore=\"(\\d+)\")(?=.*\\bFavoriteCount=\"(\\d+)\")?(?=.*\\bTags=\"(&lt\;.*gt\;)\")?.*')
LOCATION '/data/stackexchange1000/posts';

Overwriting external_table.hql


In [18]:
%%writefile sample.hql

set hive.exec.dynamic.partition.mode=nonstrict;

USE stackoverflow;

DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
    id INT,
    post_type_id TINYINT,
    owner_user_id INT,
    parent_id INT,
    score INT,
    favorite_count INT,
    tags ARRAY<STRING>
    
) 
PARTITIONED BY (year STRING, month STRING)
STORED AS SEQUENCEFILE;

FROM posts_sample_external se
INSERT OVERWRITE TABLE posts_sample PARTITION(year, month)
    select se.id, se.post_type_id, se.owner_user_id, se.parent_id, se.score, se.favorite_count,split(se.tags,"&lt\;|&gt\;"),substr(se.date,0,4), substr(se.date,6,2)
    WHERE se.id IS NOT NULL;


Overwriting sample.hql


In [19]:
%%writefile count.hql

INSERT OVERWRITE LOCAL DIRECTORY 'result' row format delimited fields terminated by '\t' stored as textfile
SELECT year,concat(year, "-", month),count(*) FROM posts_sample group by year,month LIMIT 3;

Overwriting count.hql


In [20]:
!hive -f create_db.hql
!hive -f external_table.hql
!hive -f sample.hql
!hive -f count.hql
!cat result/*|head -1


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.466 seconds
OK
Time taken: 0.137 seconds

Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.413 seconds
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 0.044 seconds
OK
Time taken: 0.356 seconds

Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.394 seconds
OK
Time taken: 0.045 seconds
OK
Time taken: 0.378 seconds
Query ID = jovyan_20171227114646_df34604d-e40e-41d8-bdea-7d61

Partition stackoverflow.posts_sample{year=2013, month=08} stats: [numFiles=1, numRows=518, totalSize=29183, rawDataSize=22082]
Partition stackoverflow.posts_sample{year=2013, month=09} stats: [numFiles=1, numRows=490, totalSize=27750, rawDataSize=21033]
Partition stackoverflow.posts_sample{year=2013, month=10} stats: [numFiles=1, numRows=554, totalSize=30428, rawDataSize=22859]
Partition stackoverflow.posts_sample{year=2013, month=11} stats: [numFiles=1, numRows=530, totalSize=29600, rawDataSize=22343]
Partition stackoverflow.posts_sample{year=2013, month=12} stats: [numFiles=1, numRows=497, totalSize=27948, rawDataSize=21140]
Partition stackoverflow.posts_sample{year=2014, month=01} stats: [numFiles=1, numRows=576, totalSize=32821, rawDataSize=24946]
Partition stackoverflow.posts_sample{year=2014, month=02} stats: [numFiles=1, numRows=569, totalSize=32167, rawDataSize=24383]
Partition stackoverflow.posts_sample{year=2014, month=03} stats: [numFiles=1, numRows=601, totalSize=33842, raw

In [93]:
%%writefile rank.hql
SELECT ps2016.tag, ps2016.popularity, ps2009.popularity,     
    rank() OVER(ORDER BY ps2009.popularity desc) r2009 from
(SELECT tag, count(*) popularity from posts_sample ps 
    LATERAL VIEW EXPLODE(tags) ps as tag 
    WHERE post_type_id=1 and year = '2016'
    GROUP BY tag) ps2016 LEFT JOIN
(SELECT tag, count(*) popularity from posts_sample
    LATERAL VIEW EXPLODE(tags) posts_sample as tag
    WHERE post_type_id=1 and year = '2009' 
    GROUP BY tag) ps2009 on ps2016.tag = ps2009.tag ORDER BY ps2016.popularity DESC LIMIT 10;

Overwriting rank.hql


In [94]:
!hive -f rank.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Query ID = jovyan_20171227153333_3c250ef2-5c61-4e87-93bc-f5a23b1697ef
Total jobs = 6
Launching Job 1 out of 6
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1514373326796_0060, Tracking URL = http://38ccdf094dbb:8088/proxy/application_1514373326796_0060/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1514373326796_0060
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2017-12-27 15:33:20,753 Stage-1 map = 0%,  reduce = 0%
2017-12-27 15:33:25,009 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 